In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For data visualization
import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns; sns.set()
color = sns.color_palette()

import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()

import warnings
warnings.simplefilter("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def read_csv(path):
    """The function reads a csv file, converts it to a data frame and returns a copy of the data frame."""
    data = pd.read_csv(path, encoding = 'unicode_escape')
    return data.copy()

df_columns_description = read_csv("/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv")
df_application_train = read_csv("/kaggle/input/home-credit-default-risk/application_train.csv")
df_application_test = read_csv("/kaggle/input/home-credit-default-risk/application_test.csv")
df_previous_application = read_csv("/kaggle/input/home-credit-default-risk/previous_application.csv")
df_POS_CASH_balance = read_csv("/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv")
df_credit_card_balance = read_csv("/kaggle/input/home-credit-default-risk/credit_card_balance.csv")
df_installments_payments = read_csv("/kaggle/input/home-credit-default-risk/installments_payments.csv")
df_bureau = read_csv("/kaggle/input/home-credit-default-risk/bureau.csv")
df_bureau_balance = read_csv("/kaggle/input/home-credit-default-risk/bureau_balance.csv")
df_sample_submission = read_csv("/kaggle/input/home-credit-default-risk/sample_submission.csv")

In [ ]:
df_list = [df_bureau, df_bureau_balance, df_previous_application, df_POS_CASH_balance,df_credit_card_balance, df_installments_payments]
for i in df_list:
    print(i.info())

## Summary Info
- Bureau
    - RangeIndex: 1,716,428 entries, 0 to 1,716,427
    - Data columns (total 17 columns)
    - dtypes: float64(8), int64(6), object(3)
- Bureau_balance
    - RangeIndex: 27,299,925 entries, 0 to 27,299,924
    - Data columns (total 3 columns)
    - dtypes: int64(2), object(1)
- Previous_application
    - RangeIndex: 1,670,214 entries, 0 to 1,670,213
    - Data columns (total 37 columns)
    - dtypes: float64(15), int64(6), object(16)
- POS_CASH_balance
    - RangeIndex: 10,001,358 entries, 0 to 10,001,357
    - Data columns (total 8 columns)
    - float64(2), int64(5), object(1)
- Credit_card_balance
    - RangeIndex: 3,840,312 entries, 0 to 3,840,311
    - Data columns (total 23 columns)
    - dtypes: float64(15), int64(7), object(1)
- Installments_balance
    - RangeIndex: 13,605,401 entries, 0 to 13,605,400
    - Data columns (total 8 columns)
    - dtypes: float64(5), int64(3)

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

## Bureau

In [ ]:
df_bureau.head(20)

In [ ]:
df_bureau.describe().T

In [ ]:
len(df_bureau.SK_ID_CURR.unique())

In [ ]:
len(df_bureau.SK_ID_BUREAU.unique())

In [ ]:
len(df_bureau_balance.SK_ID_BUREAU.unique())

In [ ]:
df_bureau.AMT_CREDIT_MAX_OVERDUE[df_bureau.AMT_CREDIT_MAX_OVERDUE!=0].value_counts().sum()

In [ ]:
df_bureau.AMT_CREDIT_SUM_OVERDUE[df_bureau.AMT_CREDIT_SUM_OVERDUE!=0].value_counts().sum()

In [ ]:
total_nan = df_bureau.isnull().sum().sort_values(ascending = False)
percent_nan = (df_bureau.isnull().sum()/df_bureau.isnull().count()*100).sort_values(ascending = False)
missing_bureau_data  = pd.concat([total_nan, percent_nan], axis=1, keys=['Total_nan', 'Percent_nan'])
missing_bureau_data.head(8)

In [ ]:
!pip install missingno
import missingno as msno
msno.bar(df_bureau);

In [ ]:
msno.matrix(df_bureau);

In [ ]:
msno.heatmap(df_bureau);

### Credit Card - AMT_CREDIT_SUM_LIMIT

In [ ]:
df_bureau.AMT_CREDIT_SUM_LIMIT[df_bureau.CREDIT_TYPE=='Credit card'].isnull().sum()

In [ ]:
df_bureau.AMT_CREDIT_SUM_LIMIT[df_bureau.CREDIT_TYPE!='Credit card'].isnull().sum()

In [ ]:
df_bureau.CREDIT_TYPE[(df_bureau.CREDIT_TYPE=='Credit card')&(df_bureau.AMT_CREDIT_SUM_LIMIT==0)].count()

In [ ]:
df_bureau[(df_bureau.CREDIT_TYPE=='Credit card')&(df_bureau.AMT_CREDIT_SUM_LIMIT==0)].head()

In [ ]:
df_bureau.CREDIT_TYPE[(df_bureau.CREDIT_TYPE=='Credit card')&(df_bureau.AMT_CREDIT_SUM_LIMIT==0)&(df_bureau.AMT_CREDIT_SUM_DEBT!=0)].count()

In [ ]:
df_bureau[(df_bureau.CREDIT_TYPE=='Credit card')&(df_bureau.AMT_CREDIT_SUM_LIMIT==0)&(df_bureau.AMT_CREDIT_SUM_DEBT!=0)].head()

In [ ]:
df_bureau[(df_bureau.CREDIT_TYPE!='Credit card')&(df_bureau.AMT_CREDIT_SUM_LIMIT!=0)].count()

In [ ]:
# for i in df_bureau.columns:
#     print(i)
#     print(df_bureau[i][df_bureau[i]=="XNA"].count())
#     print(df_bureau[i][df_bureau[i]=="XAP"].count())

In [ ]:
df_bureau.CREDIT_ACTIVE.value_counts()

In [ ]:
df_bureau.CREDIT_CURRENCY.value_counts()

In [ ]:
df_bureau.CREDIT_TYPE.value_counts()

In [ ]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
temp = df_bureau["CREDIT_ACTIVE"].value_counts()
temp.iplot(kind='bar', xTitle = 'Credit Status', yTitle = "Count", title = 'CREDIT_ACTIVE', color = 'red')

In [ ]:
plt.subplots(1,1)
sns.countplot(df_bureau.CREDIT_ACTIVE)
plt.title("CREDIT_ACTIVE",color = 'blue',fontsize=15)
plt.show()

plt.subplots(1,1)
sns.countplot(df_bureau.CREDIT_CURRENCY)
plt.title("CREDIT_CURRENCY",color = 'blue',fontsize=15)
plt.show()

plt.subplots(1,1)
sns.countplot(df_bureau.CREDIT_TYPE)
plt.xticks(rotation= 90)
plt.title("CREDIT_TYPE",color = 'blue',fontsize=15)
plt.show()

In [ ]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
temp = df_bureau["CREDIT_ACTIVE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='Credit Status', hole = 0.5)

In [ ]:
temp = df_bureau["CREDIT_CURRENCY"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='Credit Status', hole = 0.5)

In [ ]:
temp = df_bureau["CREDIT_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='Credit Type', hole = 0.5)

In [ ]:
df_bureau.groupby("CREDIT_ACTIVE").aggregate(['min', 'mean', np.median, 'max', 'size'])

In [ ]:
df_bureau.groupby("CREDIT_CURRENCY").aggregate(['min', 'mean', np.median, 'max', 'size'])

In [ ]:
df_bureau.groupby("CREDIT_TYPE").aggregate(['min', 'mean', np.median, 'max', 'size'])

In [ ]:
df_bureau.DAYS_CREDIT.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.DAYS_CREDIT, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.DAYS_CREDIT, bins = 20, kde = True);

In [ ]:
df_bureau.CREDIT_DAY_OVERDUE.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.CREDIT_DAY_OVERDUE, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.CREDIT_DAY_OVERDUE.notnull(), bins = 20, kde = True);

In [ ]:
df_bureau.DAYS_CREDIT_ENDDATE.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.DAYS_CREDIT_ENDDATE, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.DAYS_CREDIT_ENDDATE, bins = 20, kde = True);

In [ ]:
df_bureau.DAYS_ENDDATE_FACT.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.DAYS_ENDDATE_FACT, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.DAYS_ENDDATE_FACT.notnull(), bins = 20, kde = True);

In [ ]:
df_bureau.AMT_CREDIT_MAX_OVERDUE.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.AMT_CREDIT_MAX_OVERDUE, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.AMT_CREDIT_MAX_OVERDUE.notnull(), bins = 20, kde = True);

In [ ]:
df_bureau.CNT_CREDIT_PROLONG.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.CNT_CREDIT_PROLONG, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.CNT_CREDIT_PROLONG.notnull(), bins = 20, kde = True);

In [ ]:
df_bureau.AMT_CREDIT_SUM.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.AMT_CREDIT_SUM, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.AMT_CREDIT_SUM, bins = 20, kde = True);

In [ ]:
df_bureau.AMT_CREDIT_SUM_DEBT.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.AMT_CREDIT_SUM_DEBT, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.AMT_CREDIT_SUM_DEBT, bins = 20, kde = True);

In [ ]:
df_bureau.AMT_CREDIT_SUM_LIMIT.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.AMT_CREDIT_SUM_LIMIT, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.AMT_CREDIT_SUM_LIMIT.notnull(), bins = 20, kde = True);

In [ ]:
df_bureau.AMT_CREDIT_SUM_OVERDUE.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.AMT_CREDIT_SUM_OVERDUE, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.AMT_CREDIT_SUM_OVERDUE.notnull(), bins = 20, kde = True);

In [ ]:
df_bureau.DAYS_CREDIT_UPDATE.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.DAYS_CREDIT_UPDATE, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.DAYS_CREDIT_UPDATE.notnull(), bins = 20, kde = True);

In [ ]:
df_bureau.AMT_ANNUITY.value_counts().head(10)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau.AMT_ANNUITY, fit=norm, kde=False)

In [ ]:
sns.distplot(df_bureau.AMT_ANNUITY.notnull(), bins = 20, kde = True);

In [ ]:
df_bureau.head()

In [ ]:
f,ax = plt.subplots(figsize=(15, 10))
sns.heatmap(df_bureau.corr(), annot=True, linewidths=0.5, linecolor="red", fmt= '.3f',ax=ax)
plt.show()

In [ ]:
df_bureau_num = df_bureau.select_dtypes(exclude='object')
df_bureau_plot = df_bureau_num.dropna().loc[:100000, :]
sns.pairplot(df_bureau_plot,kind='reg')
plt.show()

## Bureau Balance

In [ ]:
df_bureau_balance.head(10)

In [ ]:
df_bureau_balance.MONTHS_BALANCE.isnull().sum()

In [ ]:
df_bureau_balance.STATUS.isnull().sum()

In [ ]:
df_bureau_balance.describe().T

In [ ]:
sns.countplot(df_bureau_balance.STATUS.head(100000))
plt.xticks(rotation= 90)
plt.title("STATUS",color = 'blue',fontsize=15)
plt.show()

In [ ]:
temp = df_bureau_balance["STATUS"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='STATUS', hole = 0.5)

In [ ]:
sns.distplot(df_bureau_balance.MONTHS_BALANCE, bins = 20, kde = True);

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df_bureau_balance.MONTHS_BALANCE, fit=norm, kde=False)

In [ ]:
df_bureau_balance.groupby('STATUS').aggregate(['min', 'mean', np.median, 'max'])

## Previous Application

In [ ]:
df_previous_application.describe().T

In [ ]:
df_previous_application.DAYS_FIRST_DRAWING[df_previous_application.DAYS_FIRST_DRAWING==365243].value_counts()

In [ ]:
df_previous_application.DAYS_FIRST_DUE[df_previous_application.DAYS_FIRST_DUE==365243].value_counts()

In [ ]:
df_previous_application.DAYS_LAST_DUE_1ST_VERSION[df_previous_application.DAYS_LAST_DUE_1ST_VERSION==365243].value_counts()

In [ ]:
df_previous_application.DAYS_LAST_DUE[df_previous_application.DAYS_LAST_DUE==365243].value_counts()

In [ ]:
df_previous_application.DAYS_TERMINATION[df_previous_application.DAYS_TERMINATION==365243].value_counts()

In [ ]:
df_previous_application.replace(365243,np.nan,inplace=True)

In [ ]:
df_previous_application.describe().T

In [ ]:
total_nan = df_previous_application.isnull().sum().sort_values(ascending = False)
percent_nan = (df_previous_application.isnull().sum()/df_previous_application.isnull().count()*100).sort_values(ascending = False)
missing_previous_data = pd.concat([total_nan, percent_nan], axis=1, keys=['Total_nan', 'Percent_nan'])
missing_previous_data.head(16)

In [ ]:
import missingno as msno
msno.bar(df_previous_application);

In [ ]:
msno.matrix(df_previous_application);

In [ ]:
msno.heatmap(df_previous_application);

In [ ]:
for i in df_previous_application.columns:
    print(i)
    print(df_previous_application[i][df_previous_application[i]=="XNA"].count())
    print(df_previous_application[i][df_previous_application[i]=="XAP"].count())

In [ ]:
df_previous_application.NAME_CONTRACT_TYPE.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_CONTRACT_TYPE)
plt.title("NAME_CONTRACT_TYPE",color = 'blue',fontsize=15)
plt.show()

In [ ]:
temp = df_previous_application["NAME_CONTRACT_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='CONTRACT_TYPE', hole = 0.5)

In [ ]:
df_previous_application.WEEKDAY_APPR_PROCESS_START.value_counts()

In [ ]:
sns.countplot(df_previous_application.WEEKDAY_APPR_PROCESS_START)
plt.title("WEEKDAY_APPR_PROCESS_START",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["WEEKDAY_APPR_PROCESS_START"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='WEEKDAY_APPR_PROCESS_START', hole = 0.5)

In [ ]:
df_previous_application.FLAG_LAST_APPL_PER_CONTRACT.value_counts()

In [ ]:
df_previous_application.NFLAG_LAST_APPL_IN_DAY.value_counts()

In [ ]:
df_previous_application.NAME_CASH_LOAN_PURPOSE.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_CASH_LOAN_PURPOSE)
plt.title("NAME_CASH_LOAN_PURPOSE",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_CASH_LOAN_PURPOSE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='CASH_LOAN_PURPOSE', hole = 0.5)

In [ ]:
df_previous_application.NAME_CONTRACT_STATUS.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_CONTRACT_STATUS)
plt.title("NAME_CONTRACT_STATUS",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_CONTRACT_STATUS"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='CONTRACT_STATUS', hole = 0.5)

In [ ]:
df_previous_application.NAME_PAYMENT_TYPE.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_PAYMENT_TYPE)
plt.title("NAME_PAYMENT_TYPE",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_PAYMENT_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='PAYMENT_TYPE', hole = 0.5)

In [ ]:
df_previous_application.CODE_REJECT_REASON.value_counts()

In [ ]:
sns.countplot(df_previous_application.CODE_REJECT_REASON)
plt.title("NAME_PAYMENT_TYPE",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["CODE_REJECT_REASON"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='CODE_REJECT_REASON', hole = 0.5)

In [ ]:
df_previous_application.NAME_TYPE_SUITE.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_TYPE_SUITE)
plt.title("NAME_TYPE_SUITE",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_TYPE_SUITE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='NAME_TYPE_SUITE', hole = 0.5)

In [ ]:
df_previous_application.NAME_CLIENT_TYPE.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_CLIENT_TYPE)
plt.title("NAME_CLIENT_TYPE",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_CLIENT_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='NAME_CLIENT_TYPE', hole = 0.5)

In [ ]:
df_previous_application.NAME_GOODS_CATEGORY.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_GOODS_CATEGORY)
plt.title("NAME_GOODS_CATEGORY",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_GOODS_CATEGORY"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='GOODS_CATEGORY', hole = 0.5)

In [ ]:
df_previous_application.NAME_PORTFOLIO.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_PORTFOLIO)
plt.title("NAME_PORTFOLIO",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_PORTFOLIO"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='PORTFOLIO', hole = 0.5)

In [ ]:
df_previous_application.NAME_PRODUCT_TYPE.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_PRODUCT_TYPE)
plt.title("NAME_PRODUCT_TYPE",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_PRODUCT_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='PRODUCT_TYPE', hole = 0.5)

In [ ]:
df_previous_application.CHANNEL_TYPE.value_counts()

In [ ]:
sns.countplot(df_previous_application.CHANNEL_TYPE)
plt.title("CHANNEL_TYPE",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["CHANNEL_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='CHANNEL_TYPE', hole = 0.5)

In [ ]:
df_previous_application.NAME_SELLER_INDUSTRY.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_SELLER_INDUSTRY)
plt.title("NAME_SELLER_INDUSTRY",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_SELLER_INDUSTRY"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='SELLER_INDUSTRY', hole = 0.5)

In [ ]:
df_previous_application.NAME_YIELD_GROUP.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_YIELD_GROUP)
plt.title("NAME_YIELD_GROUP",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["NAME_YIELD_GROUP"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='YIELD_GROUP', hole = 0.5)

In [ ]:
df_previous_application.PRODUCT_COMBINATION.value_counts()

In [ ]:
sns.countplot(df_previous_application.NAME_YIELD_GROUP)
plt.title("PRODUCT_COMBINATION",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_previous_application["PRODUCT_COMBINATION"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='PRODUCT_COMBINATION', hole = 0.5)

In [ ]:
df_previous_application.AMT_ANNUITY.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.AMT_ANNUITY, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.AMT_ANNUITY, fit=norm, kde=False)

In [ ]:
df_previous_application.AMT_APPLICATION.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.AMT_APPLICATION, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.AMT_APPLICATION, fit=norm, kde=False)

In [ ]:
df_previous_application.AMT_CREDIT.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.AMT_CREDIT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.AMT_CREDIT, fit=norm, kde=False)

In [ ]:
df_previous_application.AMT_DOWN_PAYMENT.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.AMT_DOWN_PAYMENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.AMT_DOWN_PAYMENT, fit=norm, kde=False)

In [ ]:
df_previous_application.AMT_GOODS_PRICE.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.AMT_GOODS_PRICE, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.AMT_GOODS_PRICE, fit=norm, kde=False)

In [ ]:
df_previous_application.HOUR_APPR_PROCESS_START.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.HOUR_APPR_PROCESS_START, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.HOUR_APPR_PROCESS_START, fit=norm, kde=False)

In [ ]:
df_previous_application.HOUR_APPR_PROCESS_START.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.HOUR_APPR_PROCESS_START, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.HOUR_APPR_PROCESS_START, fit=norm, kde=False)

In [ ]:
df_previous_application.RATE_DOWN_PAYMENT.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.RATE_DOWN_PAYMENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.RATE_DOWN_PAYMENT, fit=norm, kde=False)

In [ ]:
df_previous_application.SELLERPLACE_AREA.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.SELLERPLACE_AREA, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.SELLERPLACE_AREA, fit=norm, kde=False)

In [ ]:
df_previous_application.CNT_PAYMENT.value_counts().head(10)

In [ ]:
sns.distplot(df_previous_application.CNT_PAYMENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_previous_application.CNT_PAYMENT, fit=norm, kde=False)

In [ ]:
df_previous_application.NFLAG_INSURED_ON_APPROVAL.value_counts()

In [ ]:
f,ax = plt.subplots(figsize=(15, 10))
sns.heatmap(df_previous_application.corr(), annot=True, linewidths=0.5, linecolor="red", fmt= '.2f',ax=ax)
plt.show()

## POS_CASH Balance

In [ ]:
df_POS_CASH_balance.head(20)

In [ ]:
df_POS_CASH_balance.describe().T

In [ ]:
total_nan = df_POS_CASH_balance.isnull().sum().sort_values(ascending = False)
percent_nan = (df_POS_CASH_balance.isnull().sum()/df_POS_CASH_balance.isnull().count()*100).sort_values(ascending = False)
missing_pos_cash_data  = pd.concat([total_nan, percent_nan], axis=1, keys=['Total_nan', 'Percent_nan'])
missing_pos_cash_data

In [ ]:
msno.bar(df_POS_CASH_balance);

In [ ]:
msno.matrix(df_POS_CASH_balance);

In [ ]:
msno.heatmap(df_POS_CASH_balance);

In [ ]:
for i in df_POS_CASH_balance.columns:
    print(i)
    print(df_POS_CASH_balance[i][df_POS_CASH_balance[i]=="XNA"].count())
    print(df_POS_CASH_balance[i][df_POS_CASH_balance[i]=="XAP"].count())

In [ ]:
df_POS_CASH_balance.NAME_CONTRACT_STATUS.value_counts()

In [ ]:
sns.countplot(df_POS_CASH_balance.NAME_CONTRACT_STATUS)
plt.title("NAME_CONTRACT_STATUS",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_POS_CASH_balance["NAME_CONTRACT_STATUS"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='NAME_CONTRACT_STATUS', hole = 0.5)

In [ ]:
df_POS_CASH_balance.MONTHS_BALANCE.value_counts().head(10)

In [ ]:
sns.distplot(df_POS_CASH_balance.MONTHS_BALANCE, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_POS_CASH_balance.MONTHS_BALANCE, fit=norm, kde=False)

In [ ]:
df_POS_CASH_balance.CNT_INSTALMENT.value_counts().head(10)

In [ ]:
sns.distplot(df_POS_CASH_balance.CNT_INSTALMENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_POS_CASH_balance.CNT_INSTALMENT, fit=norm, kde=False)

In [ ]:
df_POS_CASH_balance.CNT_INSTALMENT_FUTURE.value_counts().head(10)

In [ ]:
sns.distplot(df_POS_CASH_balance.CNT_INSTALMENT_FUTURE, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_POS_CASH_balance.CNT_INSTALMENT_FUTURE, fit=norm, kde=False)

In [ ]:
df_POS_CASH_balance.SK_DPD.value_counts().head(10)

In [ ]:
sns.distplot(df_POS_CASH_balance.SK_DPD.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_POS_CASH_balance.SK_DPD, fit=norm, kde=False)

In [ ]:
df_POS_CASH_balance.SK_DPD_DEF.value_counts().head(10)

In [ ]:
sns.distplot(df_POS_CASH_balance.SK_DPD_DEF.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_POS_CASH_balance.SK_DPD_DEF, fit=norm, kde=False)

In [ ]:
f,ax = plt.subplots(figsize=(10, 5))
sns.heatmap(df_POS_CASH_balance.corr(), annot=True, linewidths=0.5, linecolor="red", fmt= '.2f',ax=ax)
plt.show()

## Credit Card Balance

In [ ]:
df_credit_card_balance.head(20)

In [ ]:
df_credit_card_balance.describe().T

In [ ]:
total_nan = df_credit_card_balance.isnull().sum().sort_values(ascending = False)
percent_nan = (df_credit_card_balance.isnull().sum()/df_credit_card_balance.isnull().count()*100).sort_values(ascending = False)
missing_credit_card  = pd.concat([total_nan, percent_nan], axis=1, keys=['Total_nan', 'Percent_nan'])
missing_credit_card.head(10)

In [ ]:
msno.bar(df_credit_card_balance);

In [ ]:
msno.matrix(df_credit_card_balance);

In [ ]:
msno.heatmap(df_credit_card_balance);

In [ ]:
# for i in df_credit_card_balance.columns:
#     print(i)
#     print(df_credit_card_balance[i][df_credit_card_balance[i]=="XNA"].count())
#     print(df_credit_card_balance[i][df_credit_card_balance[i]=="XAP"].count())

In [ ]:
df_credit_card_balance.NAME_CONTRACT_STATUS.value_counts()

In [ ]:
sns.countplot(df_credit_card_balance.NAME_CONTRACT_STATUS)
plt.title("NAME_CONTRACT_STATUS",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
temp = df_credit_card_balance["NAME_CONTRACT_STATUS"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='CONTRACT_STATUS', hole = 0.5)

In [ ]:
df_credit_card_balance.MONTHS_BALANCE.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.MONTHS_BALANCE, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.MONTHS_BALANCE, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_BALANCE.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.AMT_BALANCE, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_BALANCE, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_CREDIT_LIMIT_ACTUAL.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.AMT_CREDIT_LIMIT_ACTUAL, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_CREDIT_LIMIT_ACTUAL, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_DRAWINGS_ATM_CURRENT.value_counts().head(10)

In [ ]:
# sns.distplot(df_credit_card_balance.AMT_DRAWINGS_ATM_CURRENT.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_DRAWINGS_ATM_CURRENT, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_DRAWINGS_CURRENT.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.AMT_DRAWINGS_CURRENT.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_DRAWINGS_CURRENT, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_DRAWINGS_OTHER_CURRENT.value_counts().head(10)

In [ ]:
# sns.distplot(df_credit_card_balance.AMT_DRAWINGS_OTHER_CURRENT.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_DRAWINGS_OTHER_CURRENT, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_DRAWINGS_POS_CURRENT.value_counts().head(10)

In [ ]:
# sns.distplot(df_credit_card_balance.AMT_DRAWINGS_POS_CURRENT.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_DRAWINGS_POS_CURRENT, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_INST_MIN_REGULARITY.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.AMT_INST_MIN_REGULARITY, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_INST_MIN_REGULARITY, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_PAYMENT_CURRENT.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.AMT_PAYMENT_CURRENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_INST_MIN_REGULARITY, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_PAYMENT_TOTAL_CURRENT.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.AMT_PAYMENT_TOTAL_CURRENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_PAYMENT_TOTAL_CURRENT, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_RECEIVABLE_PRINCIPAL.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.AMT_RECEIVABLE_PRINCIPAL, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_RECEIVABLE_PRINCIPAL, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_RECIVABLE.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.AMT_RECIVABLE, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_RECIVABLE, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.AMT_TOTAL_RECEIVABLE.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.AMT_TOTAL_RECEIVABLE, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.AMT_TOTAL_RECEIVABLE, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.CNT_DRAWINGS_ATM_CURRENT.value_counts().head(10)

In [ ]:
# sns.distplot(df_credit_card_balance.CNT_DRAWINGS_ATM_CURRENT.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.CNT_DRAWINGS_ATM_CURRENT, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.CNT_DRAWINGS_CURRENT.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.CNT_DRAWINGS_CURRENT.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.CNT_DRAWINGS_CURRENT, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.CNT_DRAWINGS_OTHER_CURRENT.value_counts().head(10)

In [ ]:
# sns.distplot(df_credit_card_balance.CNT_DRAWINGS_OTHER_CURRENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.CNT_DRAWINGS_OTHER_CURRENT, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.CNT_DRAWINGS_POS_CURRENT.value_counts().head(10)

In [ ]:
# sns.distplot(df_credit_card_balance.CNT_DRAWINGS_POS_CURRENT.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.CNT_DRAWINGS_POS_CURRENT, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.CNT_INSTALMENT_MATURE_CUM.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.CNT_INSTALMENT_MATURE_CUM, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.CNT_INSTALMENT_MATURE_CUM, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.SK_DPD.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.SK_DPD.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.SK_DPD_DEF, fit=norm, kde=False)

In [ ]:
df_credit_card_balance.SK_DPD_DEF.value_counts().head(10)

In [ ]:
sns.distplot(df_credit_card_balance.SK_DPD_DEF.notnull(), bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_credit_card_balance.SK_DPD_DEF, fit=norm, kde=False)

In [ ]:
f,ax = plt.subplots(figsize=(15, 10))
sns.heatmap(df_credit_card_balance.corr(), annot=True, linewidths=0.5, linecolor="red", fmt= '.2f',ax=ax)
plt.show()

## Installments Payments

In [ ]:
df_installments_payments.head(20)

In [ ]:
df_installments_payments.describe().T

In [ ]:
total_nan = df_installments_payments.isnull().sum().sort_values(ascending = False)
percent_nan = (df_installments_payments.isnull().sum()/df_installments_payments.isnull().count()*100).sort_values(ascending = False)
missing_installments  = pd.concat([total_nan, percent_nan], axis=1, keys=['Total_nan', 'Percent_nan'])
missing_installments

In [ ]:
msno.bar(df_installments_payments);

In [ ]:
msno.matrix(df_installments_payments);

In [ ]:
msno.heatmap(df_installments_payments);

In [ ]:
# for i in df_installments_payments.columns:
#     print(i)
#     print(df_installments_payments[i][df_installments_payments[i]=="XNA"].count())
#     print(df_installments_payments[i][df_installments_payments[i]=="XAP"].count())

In [ ]:
df_installments_payments.NUM_INSTALMENT_VERSION.value_counts().head(10)

In [ ]:
sns.distplot(df_installments_payments.NUM_INSTALMENT_VERSION, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_installments_payments.NUM_INSTALMENT_VERSION, fit=norm, kde=False)

In [ ]:
df_installments_payments.NUM_INSTALMENT_NUMBER.value_counts().head(10)

In [ ]:
sns.distplot(df_installments_payments.NUM_INSTALMENT_NUMBER, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_installments_payments.NUM_INSTALMENT_NUMBER, fit=norm, kde=False)

In [ ]:
df_installments_payments.DAYS_INSTALMENT.value_counts().head(10)

In [ ]:
sns.distplot(df_installments_payments.DAYS_INSTALMENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_installments_payments.DAYS_INSTALMENT, fit=norm, kde=False)

In [ ]:
df_installments_payments.DAYS_ENTRY_PAYMENT.value_counts().head(10)

In [ ]:
sns.distplot(df_installments_payments.DAYS_ENTRY_PAYMENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_installments_payments.DAYS_ENTRY_PAYMENT, fit=norm, kde=False)

In [ ]:
df_installments_payments.AMT_INSTALMENT.value_counts().head(10)

In [ ]:
sns.distplot(df_installments_payments.AMT_INSTALMENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_installments_payments.AMT_INSTALMENT, fit=norm, kde=False)

In [ ]:
df_installments_payments.AMT_PAYMENT.value_counts().head(10)

In [ ]:
sns.distplot(df_installments_payments.AMT_PAYMENT, bins = 20, kde = True);

In [ ]:
ax = sns.distplot(df_installments_payments.AMT_PAYMENT, fit=norm, kde=False)

In [ ]:
f,ax = plt.subplots(figsize=(10, 5))
sns.heatmap(df_installments_payments.corr(), annot=True, linewidths=0.5, linecolor="red", fmt= '.2f',ax=ax)
plt.show()

In [ ]:
# pd.reset_option("display.max_rows")
# pd.reset_option("display.max_columns")
# pd.get_option("display.max_rows")